In [ ]:
import urllib
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from datetime import date, datetime, timedelta
from jugaad_data.nse import full_bhavcopy_save, bhavcopy_fo_save, bhavcopy_raw
import time
import os


ERROR_TABLE_NAME = "nse_error_dates"
HISTORY_TABLE = "Nse_history"
BHAVCOPY_FOLDER = "nse_bhavcopies"

HOST = "DESKTOP-0H6SQRG"
DB = "nse"
DRIVER = "ODBC+Driver+11+for+SQL+Server"
# conn = create_engine(f'mssql+pyodbc://{HOST}/{DB}?trusted_connection=yes&driver={DRIVER}')


CREATE_ERROR_TABLE_QUERY = f"""IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{ERROR_TABLE_NAME}' and xtype='U')
                                create table {ERROR_TABLE_NAME}(id int primary key identity(1,1), [date] date not null,
                                timestamp datetime default current_timestamp)"""

REQUIRED_COLUMNS = ["Date", "Symbol", "Series", "Prev Close", "Open", "High", "Low", "Last", "Close", "VWAP",
                    "Volume", "Trades", "Deliverable Volume", "%Deliverble"]


CREATE_HISTORY_TABLE_QUERY = f"""IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{HISTORY_TABLE}' and xtype='U')
                                CREATE TABLE {HISTORY_TABLE}(
                                [id] [bigint] primary key identity(1,1),
                                [Date] [date] NULL,
                                [Symbol] [varchar](50) NULL,
                                [Series] [varchar](50) NULL,
                                [Prev Close] [varchar](50) NULL,
                                [Open] [varchar](50) NULL,
                                [High] [varchar](50) NULL,
                                [Low] [varchar](50) NULL,
                                [Last] [varchar](50) NULL,
                                [Close] [varchar](50) NULL,
                                [VWAP] [varchar](50) NULL,
                                [Volume] [varchar](50) NULL,
                                [Turnover] [varchar](50) NULL,
                                [Trades] [varchar](50) NULL,
                                [Deliverable Volume] [varchar](50) NULL,
                                [%Deliverble] [varchar](50) NULL,
                                [timestamp] [datetime] default current_timestamp
                            )"""

column_mappings = {
    "date": "Date", "symbol": "Symbol", "pclose": "Prev Close", "open": "Open", "high": "High", "low":"Low", "close": "Close",
    "vwap": "VWAP", "volume": "Volume", "trades": "Trades", "dvolume": "Deliverable Volume", "pdeliverble":"%Deliverble",
    "series": "Series", "last": "Last"}


# conn = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
engine = create_engine(f'mssql+pyodbc://{HOST}/{DB}?trusted_connection=yes&driver={DRIVER}')
conn = engine.connect()
realpath_bhavcopy = os.path.join(os.getcwd(), BHAVCOPY_FOLDER)
if not os.path.exists(realpath_bhavcopy):
    os.mkdir(realpath_bhavcopy)


def execute_sql(query, commit=False):
    result = conn.execute(text(query))
    if commit:
        conn.commit()
    return result


execute_sql(CREATE_ERROR_TABLE_QUERY, commit=True)
execute_sql(CREATE_HISTORY_TABLE_QUERY, commit=True)


def if_exists(tablename, date=None, df=None):
    DUPLICATE_CHECK_QUERY = """select * from {TABLENAME} where {DATE_COL}='{DATE}' 
                                and {SYMBOL_COL}='{SYMBOL}'"""
    SINGLE_CHECK_QUERY = "select * from {TABLENAME} where {DATE_COL}='{DATE}'"
    result = None
    if date:
        result = execute_sql(SINGLE_CHECK_QUERY.format(TABLENAME=tablename, DATE_COL=column_mappings["date"],
                                                           DATE=date)).fetchone()
        
    elif df is not None:
        rand_rec = df.sample()
        rand_rec = rand_rec.to_dict("records")[0]
        result = execute_sql(DUPLICATE_CHECK_QUERY.format(TABLENAME=tablename, 
                                                           DATE_COL=column_mappings["date"],
                                                           SYMBOL_COL=column_mappings["symbol"], 
                                                           DATE=rand_rec[column_mappings["date"]],
                                                           SYMBOL=rand_rec[column_mappings["symbol"]])).fetchone()
    if result:
        return True
    return False
        

def convert_strto_datetime(date_time):
    try:
        datetime_str = datetime.strptime(date_time, '%d-%b-%Y')
    except ValueError:
        datetime_str = datetime.strptime(date_time, '%d-%m-%Y')
    return datetime_str

def clean_data(df):
    df = df.rename(columns={" DATE1": column_mappings["date"], "SYMBOL": column_mappings["symbol"],
                            " SERIES": column_mappings["series"], " PREV_CLOSE": column_mappings["pclose"],
                            " OPEN_PRICE": column_mappings["open"], " HIGH_PRICE": column_mappings["high"] , 
                            " LOW_PRICE": column_mappings["low"], " LAST_PRICE": column_mappings["last"],
                            " CLOSE_PRICE": column_mappings["close"], " AVG_PRICE": column_mappings["vwap"], 
                            " TTL_TRD_QNTY": column_mappings["volume"], " NO_OF_TRADES": column_mappings["trades"],
                            " DELIV_QTY": column_mappings["dvolume"], " DELIV_PER": column_mappings["pdeliverble"]})

    df[column_mappings["date"]] = df[column_mappings["date"]].apply(lambda x: x.strip())
    df[column_mappings["date"]] = df[column_mappings["date"]].apply(convert_strto_datetime)
    df[column_mappings["symbol"]] = df[column_mappings["symbol"]].apply(lambda x: x.strip())
    df[column_mappings["series"]] = df[column_mappings["series"]].apply(lambda x: x.strip())

    return df[REQUIRED_COLUMNS]

def save_bc(dt):
    fileloc = full_bhavcopy_save(dt, realpath_bhavcopy)
    df = pd.read_csv(fileloc)
    df = clean_data(df)
    df.to_sql(HISTORY_TABLE, engine, if_exists="append", index=False)
    print(f"Saved data for {dt.strftime('%d-%b-%Y')} day..")
    return df

## download data from existing table based on last date

In [ ]:
last_date = execute_sql(f"select top(1){column_mappings['date']} from {HISTORY_TABLE} order by {column_mappings['date']} DESC").fetchone()
print(f"last date found in database is {getattr(last_date,column_mappings['date']).strftime('%d-%b-%Y')}")
next_date = getattr(last_date,column_mappings['date']
yesterday = date.today()-timedelta(days=1)
while True:
    try:
        if yesterday < next_date:
            break
        next_date = next_date+timedelta(days=1)
        str_date = next_date.strftime('%d-%b-%Y')
        if next_date.weekday() < 5:
#             print(str_date)
            try:
                if if_exists(HISTORY_TABLE, next_date):
                    print(f"Data already exists for {str_date} in {HISTORY_TABLE} table")
                    continue
                df = save_bc(next_date)
            except Exception as e:
                print(f"error occured on {str_date} {e}")
                execute_sql(f"insert into {ERROR_TABLE_NAME}([date]) values('{next_date}')", commit=True)
                continue
        time.sleep(1)
    except KeyboardInterrupt:
        print("Download interrupted..")
        break

## Download data for the date in error table

In [ ]:
err_dates = execute_sql(f"select [date] as {column_mappings['date']} from {ERROR_TABLE_NAME}").fetchall()
for d in err_dates:
    try:
#         print(d[column_mappings["date"]].strftime('%d-%b-%Y'))
        save_bc(getattr(d, column_mappings["date"]))
        execute_sql(f"delete {ERROR_TABLE_NAME} where date='{d[column_mappings['date']]}'", commit=True)
    except Exception as e:
        print(e)
    time.sleep(1)
        

## Download data for a Single day 

In [ ]:
download_date = "03-01-2022"                #dd/mm/yyyy

dt = convert_strto_datetime(download_date)
if dt.weekday() > 4:
    print(f"Sorry its a weekend.. {download_date}")
elif if_exists(HISTORY_TABLE, dt):
    print(f"Data already exists for {download_date} in {HISTORY_TABLE} table")
else:
    df = save_bc(dt)
    
    
    
    